# TODO
- Transform: 'dry_weight' and 'total_weight', add column 'dry_weight_kg' and 'total_weight_kg'. 
- Load in new Redshift table.

## Spark setup

In [24]:
from os import environ
from pyspark.sql import SparkSession

In [25]:
driver_path = environ["DRIVER_PATH"]
print(driver_path)

spark = SparkSession.builder \
        .master("local") \
        .appName("entregable-2") \
        .config("spark.jars", driver_path) \
        .config("spark.executor.extraClassPath", driver_path) \
        .getOrCreate()

/home/entregable_2/driver_jdbc/postgresql-42.6.0.jar


## Lectura de datos de la API en formato JSON 

Los datos tienen la siguiente estructura

```json
[
    {dato1},
    {dato2},
    ...,
    {datoN}
]
```

Activamos la opción [multiline](https://sparkbyexamples.com/pyspark/pyspark-read-json-file-into-dataframe/#read-json-multiline) para que Spark pueda armar el DataFrame correctamente. 

In [27]:
df = spark.read.option("multiline", "true").json("api/motorcycles.json")

## Exploración de los datos

Una posible mejora del entregable 1 era:

- El schema puede variar según el fabricante. Podríamos obtener todas las características disponibles en los datos extraídos y definir un schema con todas las columnas posibles para la tabla de Redshift en lugar de priozar un subconjunto de ellas.

La tabla creada en la entrega anterior tiene 24 columnas:

```
1 make
2 model
3 year
4 type
5 displacement
6 engine
7 power
8 top_speed
9 compression
10 bore_stroke
11 cooling
12 fuel_consumption
13 emission
14 front_suspension
15 rear_suspension
16 front_tire
17 rear_tire
18 front_brakes
19 rear_brakes
20 dry_weight
21 total_height
22 total_length
23 total_width
24 starter
```

Veamos lo que nos dice Spark.

In [46]:
df_cols = df.columns

print(f"# columnas = {len(df_cols)}")

# columnas = 41


Esto nos dice que Spark fue capaz de entender la estructura de los datos desde la carga. El DataFrame tiene un método más cómodo para visualizar el schema completo

In [38]:
df.printSchema()

root
 |-- bore_stroke: string (nullable = true)
 |-- clutch: string (nullable = true)
 |-- compression: string (nullable = true)
 |-- cooling: string (nullable = true)
 |-- displacement: string (nullable = true)
 |-- dry_weight: string (nullable = true)
 |-- emission: string (nullable = true)
 |-- engine: string (nullable = true)
 |-- frame: string (nullable = true)
 |-- front_brakes: string (nullable = true)
 |-- front_suspension: string (nullable = true)
 |-- front_tire: string (nullable = true)
 |-- front_wheel_travel: string (nullable = true)
 |-- fuel_capacity: string (nullable = true)
 |-- fuel_consumption: string (nullable = true)
 |-- fuel_control: string (nullable = true)
 |-- fuel_system: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- ground_clearance: string (nullable = true)
 |-- ignition: string (nullable = true)
 |-- lubrication: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- power: string (null

¡Genial! Ahora veamos algunos valores

In [51]:
df.show(1)

+--------------------+------+-----------+-------+--------------------+--------------------+--------------------+--------------------+-----+------------+----------------+----------+------------------+-------------+--------------------+------------+-----------+-------+----------------+--------+-----------+-------+-----------------+--------------------+--------------------+---------------+----------+-----------------+-----------+---------------+--------------------+------+--------------------+--------------------+------------+--------------------+------------+-----+-------------------+---------+----+
|         bore_stroke|clutch|compression|cooling|        displacement|          dry_weight|            emission|              engine|frame|front_brakes|front_suspension|front_tire|front_wheel_travel|fuel_capacity|    fuel_consumption|fuel_control|fuel_system|gearbox|ground_clearance|ignition|lubrication|   make|            model|               power|         rear_brakes|rear_suspension| rear_

Ilegible. Al parecer es un problema de Jupyter al formatear la tabla que imprime Spark. Probemos jugando con algunos parámetros de [show](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.DataFrame.show.html)

In [52]:
df.show(n=1, truncate=False, vertical=True)

-RECORD 0--------------------------------------------------------------
 bore_stroke         | 52.4 x 49.5 mm (2.1 x 1.9 inches)               
 clutch              | null                                            
 compression         | 8.8:1                                           
 cooling             | Air                                             
 displacement        | 110.0 ccm (6.71 cubic inches)                   
 dry_weight          | 99.0 kg (218.3 pounds)                          
 emission            | 48.7 CO2 g/km. (CO2 - Carbon dioxide emission)  
 engine              | Single cylinder, four-stroke                    
 frame               | null                                            
 front_brakes        | Single disc                                     
 front_suspension    | Telescopic fork                                 
 front_tire          | 130/60-13                                       
 front_wheel_travel  | null                                     

Mucho mejor. Vemos que algunas columnas son `null`, lo que tiene sentido porque algunos fabricantes incluyen datos que otros no.

Busquemos filas duplicadas. A la cantidad total de filas vamos a restarle la cantidad de filas distintas

In [65]:
total_rows = df.select(df_cols).count()
print(total_rows)

750


In [66]:
total_distinct_rows = df.select(df_cols).distinct().count()
print(total_distinct_rows)

750


In [67]:
repeated_rows = total_rows - total_distinct_rows
print(repeated_rows)

0


Por lo que no tenemos datos repetidos.

## Transformación de los datos

Si volvemos al schema del DataFrame vamos a notar que la columna `year` es de tipo `string`. Hagamos, por conveniencia, que sea de tipo `integer`. Para eso vamos a usar la función `col` de PySpark

In [79]:
from pyspark.sql.functions import col

transformed_df = df.withColumn("year", col("year").cast("Integer"))
print(df.schema["year"].dataType)
print(transformed_df.schema["year"].dataType)

StringType()
IntegerType()


El nuevo DataFrame `transformed_df` tiene el mismo schema que el anterior salvo por la columna `year` que ahora es un `integer`.

In [3]:
# Query Redshift using Spark SQL
query = f"select * from {environ['REDSHIFT_CODER_SCHEMA']}.motorcycles"
data = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{environ['REDSHIFT_CODER_HOST']}:{environ['REDSHIFT_CODER_PORT']}/{environ['REDSHIFT_CODER_DB']}") \
    .option("dbtable", f"({query}) as tmp_table") \
    .option("user", environ['REDSHIFT_CODER_USER']) \
    .option("password", environ['REDSHIFT_CODER_PASSWORD']) \
    .option("driver", "org.postgresql.Driver") \
    .load()